In [141]:
# ShellHacks MAIN.py

# Variable definition
ifile = 'jobs.mp3'

import speech2text
import translate_text
import synthesize_text

# 1. Call speech2tex.py
speech2text_result = speech2text(ifile)

print('1. speech2text: \r\ntext: ' + speech2text_result.results[0].alternatives[0].transcript,'\r\nconfidence: %2.2f' % speech2text_result.results[0].alternatives[0].confidence)

# 2. Call translate.py

target = 'PT_BR'
translate_text_result = translate_text(target, speech2text_result.results[0].alternatives[0].transcript)

print('2. Translate Text: ', translate_text_result['translatedText']) # dict -> ['translatedText']

# 3. Call Text2Speech

fname = 'output.mp3'
synthesize_text_result = synthesize_text(translate_text_result['translatedText'], fname)

# 4. Play Sound

# import the library
# pip install playsound
# pip install pygobject
from playsound import playsound

playsound(fname)


1. speech2text: 
text: Today, I want to tell you three stories from my life. That's it. No. Big deal. Just three stories. 
confidence: 0.97
2. Translate Text:  Hoje, quero contar a vocês três histórias da minha vida. É isso. Nada demais. Apenas três histórias.
Audio content written to file "output.mp3"


In [131]:
type(translate_text_result)
translate_text_result['translatedText']

'Hoje, quero contar a vocês três histórias da minha vida. É isso. Nada demais. Apenas três histórias.'

In [76]:
# ShellHacks SpeechtoText
# pip install --upgrade google-cloud-speech
# ffmpeg -i jobs.mp3 jobs.wav
# 01. speech2text

def speech2text(ifile):
    import os 
    from google.cloud import speech 

    # Google Cloud -> IAM & Admin -> create service Acc --> create key
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'pelagic-rig-327114-388e59ecc397.json'
    speech_client = speech.SpeechClient()

    # 1. Ex 1 local media file

    media_file_name_mp3 = ifile #'jobs.mp3'

    with open(media_file_name_mp3, 'rb') as f1: 
        byte_data_mp3 = f1.read()
    audio_mp3 = speech.RecognitionAudio(content=byte_data_mp3)

    ## 2. Configure Media files

    config_mp3 = speech.RecognitionConfig(
        sample_rate_hertz=48000, 
        enable_automatic_punctuation=True,
        language_code='en-US'
    )


    # 3. Transcibing the RecognitionAudio objects
    # needed to enable Google Cloud to Speech module on website

    response_standard_mp3 = speech_client.recognize(
        config = config_mp3,
        audio = audio_mp3
    )

#     print(response_standard_mp3)
    return(response_standard_mp3)



In [74]:
print(response_standard_mp3.results[0].alternatives[0].transcript)

Today, I want to tell you three stories from my life. That's it. No. Big deal. Just three stories.


In [3]:
type(audio_mp3)

google.cloud.speech_v1.types.cloud_speech.RecognitionAudio

In [118]:
## Google Translate
# pip install --upgrade google-cloud-translate
# google cloud likely is returning a protocol buffer, you can check out https://developers.google.com/protocol-buffers/docs/pythontutorial for details on how to work with them in python
# 2. translate.py

def translate_text(target, text):
    """Translates text into the target language.

    Target must be an ISO 639-1 language code.
    See https://g.co/cloud/translate/v2/translate-reference#supported_languages
    """
    import os 

    import six
    from google.cloud import translate_v2 as translate

    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'pelagic-rig-327114-388e59ecc397.json'
    
    
    translate_client = translate.Client()

    if isinstance(text, six.binary_type):
        text = text.decode("utf-8")

    # Text can also be a sequence of strings, in which case this method
    # will return a sequence of results for each text.
    result_tr = translate_client.translate(text, target_language=target)

#     print(u"Text: {}".format(result_tr["input"]))
#     print(u"Translation: {}".format(result_tr["translatedText"]))
#     print(u"Detected source language: {}".format(result_tr["detectedSourceLanguage"]))

    return result_tr # type dict
    
# result_tr = translate_text('PT_BR', response_standard_mp3.results[0].alternatives[0].transcript)

In [5]:
print(result_tr)
type(result_tr)
result_tr['translatedText']

{'translatedText': 'Hoje, quero contar a vocês três histórias da minha vida. É isso. Nada demais. Apenas três histórias.', 'detectedSourceLanguage': 'en', 'input': "Today, I want to tell you three stories from my life. That's it. No. Big deal. Just three stories."}


'Hoje, quero contar a vocês três histórias da minha vida. É isso. Nada demais. Apenas três histórias.'

In [138]:
## Google text to speech
# pip install --upgrade google-cloud-texttospeech
# 3. text2speech

def synthesize_text(text, fname):
    """Synthesizes speech from the input string of text."""
    from google.cloud import texttospeech

    client = texttospeech.TextToSpeechClient()

    input_text = texttospeech.SynthesisInput(text=text)

    # Note: the voice can also be specified by name.
    # Names of voices can be retrieved with client.list_voices().
    # https://cloud.google.com/text-to-speech/docs/voices
    voice = texttospeech.VoiceSelectionParams(
        #language_code="en-US",
        language_code='pt-BR',
        #name="en-US-Standard-C",
        name="	pt-BR-Standard-A",
        ssml_gender=texttospeech.SsmlVoiceGender.FEMALE,
    )

    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.MP3
    )

    response = client.synthesize_speech(
        request={"input": input_text, "voice": voice, "audio_config": audio_config}
    )

    # The response's audio_content is binary.
    with open(fname, "wb") as out:
        out.write(response.audio_content)
        print('Audio content written to file "output.mp3"')

##

#synthesize_text(response_standard_mp3.results[0].alternatives[0].transcript )
# synthesize_text(result_tr['translatedText'])

In [7]:
## Play mp3 file


In [8]:
from google.cloud import texttospeech
client = texttospeech.TextToSpeechClient()
client.list_voices()

voices {
  language_codes: "ar-XA"
  name: "ar-XA-Wavenet-A"
  ssml_gender: FEMALE
  natural_sample_rate_hertz: 24000
}
voices {
  language_codes: "ar-XA"
  name: "ar-XA-Wavenet-B"
  ssml_gender: MALE
  natural_sample_rate_hertz: 24000
}
voices {
  language_codes: "ar-XA"
  name: "ar-XA-Wavenet-C"
  ssml_gender: MALE
  natural_sample_rate_hertz: 24000
}
voices {
  language_codes: "ar-XA"
  name: "ar-XA-Wavenet-D"
  ssml_gender: FEMALE
  natural_sample_rate_hertz: 24000
}
voices {
  language_codes: "bn-IN"
  name: "bn-IN-Wavenet-A"
  ssml_gender: FEMALE
  natural_sample_rate_hertz: 24000
}
voices {
  language_codes: "bn-IN"
  name: "bn-IN-Wavenet-B"
  ssml_gender: MALE
  natural_sample_rate_hertz: 24000
}
voices {
  language_codes: "en-GB"
  name: "en-GB-Wavenet-A"
  ssml_gender: FEMALE
  natural_sample_rate_hertz: 24000
}
voices {
  language_codes: "en-GB"
  name: "en-GB-Wavenet-B"
  ssml_gender: MALE
  natural_sample_rate_hertz: 24000
}
voices {
  language_codes: "en-GB"
  name: "en-G

In [140]:
#import the library
# pip install playsound
# pip install pygobject
from playsound import playsound

playsound('output.mp3')

playsound is relying on another python subprocess. Please use `pip install pygobject` if you want playsound to run more efficiently.
